# Lab - Week2

GI67216 

**<u>refernce</u>**: for study 
##### https://www.w3schools.com/python/

**<u>refernce</u>**: for study 
##### https://pandas.pydata.org/docs/reference/frame.html

## Questions:

1. When do the first and last flights leave each day?

1. When there is a missing value in dep_time then this is an indication of canceled flight. Find the number of cancelled flights for each (carrier, origin) combination.

1. Which carrier has the worst dep_delays?

1. Which plane (tailnum) has the worst on-time record?

1. For each plane, count the number of flights before the first delay of greater than 1 hour.

1. By using the flights data find all flights:

1. Had an arrival delay of two or more hours.

1. Flew to Houston (IAH or HOU)

1. Were operated by American, Delta

1. How many values are missing in dep_time?

1. Sort flight to find fastest flight.

1. Which flights travelled the shortest?

1. Merge `flights` dataframe with `weather` dataframe and investigate if weather has any affect on delays

In [27]:
import pandas as pd

flights = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/nyc_flights.csv')

weather = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/relational_data/nyc_weather.csv')
flights.head()
weather.head()


,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
0,EWR,2013,1,1,1,39.02,26.06,59.37,270.0,10.35702,NaN,0.0,1012.0,10.0,2013-01-01T06:00:00Z
1,EWR,2013,1,1,2,39.02,26.96,61.63,250.0,8.05546,NaN,0.0,1012.3,10.0,2013-01-01T07:00:00Z
2,EWR,2013,1,1,3,39.02,28.04,64.43,240.0,11.50780,NaN,0.0,1012.5,10.0,2013-01-01T08:00:00Z
3,EWR,2013,1,1,4,39.92,28.04,62.21,250.0,12.65858,NaN,0.0,1012.2,10.0,2013-01-01T09:00:00Z
4,EWR,2013,1,1,5,39.02,28.04,64.43,260.0,12.65858,NaN,0.0,1011.9,10.0,2013-01-01T10:00:00Z


In [2]:
import pandas as pd

flights = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/nyc_flights.csv')

weather = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/relational_data/nyc_weather.csv')
flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


#### 1.When do the first and last flights leave each day?

In [16]:
first_f = flights.groupby(['year', 'month', 'day'])['dep_time'].min().reset_index()
last_f = flights.groupby(['year', 'month', 'day'])['dep_time'].max().reset_index()
merged_df = first_f.merge(last_f,how="left",on=['year', 'month', 'day'])
merged_df.columns =["Year","Month","Day","First Flight Time","Last Flight Time"]
print(f": {merged_df}")


:      Year  Month  Day  First Flight Time  Last Flight Time
0    2013      1    1              517.0            2356.0
1    2013      1    2               42.0            2354.0
2    2013      1    3               32.0            2349.0
3    2013      1    4               25.0            2358.0
4    2013      1    5               14.0            2357.0
..    ...    ...  ...                ...               ...
360  2013     12   27                2.0            2351.0
361  2013     12   28                7.0            2358.0
362  2013     12   29                3.0            2400.0
363  2013     12   30                1.0            2356.0
364  2013     12   31               13.0            2356.0

[365 rows x 5 columns]


#### 2.When there is a missing value in dep_time then this is an indication of canceled flight. Find the number of cancelled flights for each (carrier, origin) combination.

In [19]:
cancelled_f = flights[flights['dep_time'].isna()].groupby(['carrier', 'origin']).size().reset_index(name='count for cancelled flights')
print(f": {cancelled_f}")

:    carrier origin  count for cancelled flights
0       9E    EWR                           68
1       9E    JFK                          807
2       9E    LGA                          169
3       AA    EWR                           99
4       AA    JFK                          141
5       AA    LGA                          396
6       AS    EWR                            2
7       B6    EWR                           74
8       B6    JFK                          315
9       B6    LGA                           77
10      DL    EWR                           39
11      DL    JFK                          100
12      DL    LGA                          210
13      EV    EWR                         2164
14      EV    JFK                           82
15      EV    LGA                          571
16      F9    LGA                            3
17      FL    LGA                           73
18      MQ    EWR                          168
19      MQ    JFK                          327
20      MQ 

#### 3.Which carrier has the worst dep_delays?

In [20]:
worst_carrier_delay = flights.groupby('carrier')['dep_delay'].mean().idxmax()
print(f": {worst_carrier_delay}")

: F9


#### 4.Which plane (tailnum) has the worst on-time record?

In [21]:
worst_flight_r = flights.groupby('tailnum')['dep_delay'].mean().idxmax()

print(f": {worst_flight_r}")

: N844MH


#### 4.For each plane, count the number of flights before the first delay of greater than 1 hour.

In [22]:
flights['dep_delay'] = flights['dep_delay'].fillna(0)  
## I Replaced NaN with 0 for easier calculation
first_f_delay_over_1_hour = flights.groupby('tailnum')['dep_delay'].apply(lambda x: (x > 60).cumsum().eq(0).sum())

print(f": {first_f_delay_over_1_hour}")


: tailnum
D942DN     0
N0EGMQ    48
N10156     9
N102UW    33
N103US    46
          ..
N997DL    12
N998AT    15
N998DL     2
N999DN    19
N9EAMQ    26
Name: dep_delay, Length: 4043, dtype: int64


#### 5.By using the flights data find all flights and Flew to Houston (IAH or HOU) and Were operated by American, Delta

In [23]:
filtered_f = flights[(flights['arr_delay'] >= 120) &
                           (flights['dest'].isin(['IAH', 'HOU'])) &
                           (flights['carrier'].isin(['American', 'Delta']))]

print(f": {filtered_f}")


: Empty DataFrame
Columns: [year, month, day, dep_time, sched_dep_time, dep_delay, arr_time, sched_arr_time, arr_delay, carrier, flight, tailnum, origin, dest, air_time, distance, hour, minute, time_hour]
Index: []


#### 10.How many values are missing in dep_time?

In [9]:
missing_dep_time_count = flights['dep_time'].isna().sum()
print(f": {missing_dep_time_count}")


: 8255


#### 11.Sort flight to find fastest flight.

In [24]:
fastest_f = flights.loc[flights['air_time'].idxmin()]
print(f": {fastest_f}")


: year                              2013
month                                1
day                                 16
dep_time                        1355.0
sched_dep_time                    1315
dep_delay                         40.0
arr_time                        1442.0
sched_arr_time                    1411
arr_delay                         31.0
carrier                             EV
flight                            4368
tailnum                         N16911
origin                             EWR
dest                               BDL
air_time                          20.0
distance                           116
hour                                13
minute                              15
time_hour         2013-01-16T18:00:00Z
Name: 13524, dtype: object


#### 12.Which flights travelled the shortest?

In [25]:
shortest_distance_f = flights.loc[flights['distance'].idxmin()]
print(f": {shortest_distance_f}")


: year                              2013
month                                7
day                                 27
dep_time                           NaN
sched_dep_time                     106
dep_delay                          0.0
arr_time                           NaN
sched_arr_time                     245
arr_delay                          NaN
carrier                             US
flight                            1632
tailnum                            NaN
origin                             EWR
dest                               LGA
air_time                           NaN
distance                            17
hour                                 1
minute                               6
time_hour         2013-07-27T05:00:00Z
Name: 275945, dtype: object


#### 13.Merge flights dataframe with weather dataframe and investigate if weather has any affect on delays

In [26]:
merged_data_f = flights.merge(weather, on=['year', 'month', 'day', 'hour', 'origin'])
print(f": {merged_data_f}")


:         year  month  day  dep_time  sched_dep_time  dep_delay  arr_time  \
0       2013      1    1     517.0             515        2.0     830.0   
1       2013      1    1     554.0             558       -4.0     740.0   
2       2013      1    1     533.0             529        4.0     850.0   
3       2013      1    1     542.0             540        2.0     923.0   
4       2013      1    1     544.0             545       -1.0    1004.0   
...      ...    ...  ...       ...             ...        ...       ...   
335215  2013      9   30    2240.0            2245       -5.0    2334.0   
335216  2013      9   30    2240.0            2250      -10.0    2347.0   
335217  2013      9   30    2241.0            2246       -5.0    2345.0   
335218  2013      9   30    2307.0            2255       12.0    2359.0   
335219  2013      9   30    2349.0            2359      -10.0     325.0   

        sched_arr_time  arr_delay carrier  ...   temp   dewp  humid wind_dir  \
0                